# Part One

In [289]:
data class Point(val x: Int, val y: Int) {
    val adjacentPoints: List<Point> by lazy {
        buildList {
            val rangeWithBorder = (x - 1)..(x + 1)
            addAll(rangeWithBorder.map { x -> Point(x, y - 1) }) // Add the line above
            addAll(rangeWithBorder.map { x -> Point(x, y + 1) }) // Add the line below
            add(Point(x - 1, y)) // Add the point before
            add(Point(x + 1, y)) // Add the point after
        }.filter { it.x > -1 && it.y > -1 }
    }
}

data class GridNumber(val start: Point, val end: Point, val value: Int, val partial: Boolean = false) {
    val xRange: List<Point> get() = (start.x..end.x).map { x -> Point(x, start.y) }
    val adjacentPoints: List<Point> = buildList {
        val rangeWithBorder = (start.x - 1)..(end.x + 1)
        addAll(rangeWithBorder.map { x -> Point(x, start.y - 1) }) // Add the line above
        addAll(rangeWithBorder.map { x -> Point(x, start.y + 1) }) // Add the line below
        add(Point(start.x - 1, start.y)) // Add the point before
        add(Point(end.x + 1, start.y)) // Add the point after
    }.filter { it.x > -1 && it.y > -1 }

    fun overlaps(point: Point) = point.x >= start.x && point.x <= end.x && point.y == start.y
}


In [290]:
data class Grid(val map: Map<Point, Char>) : Map<Point, Char> by map {
    val numbers: List<GridNumber> =
        map.entries.fold<Map.Entry<Point, Char>, List<GridNumber>>(emptyList()) { list, entry ->
            val (point, char) = entry
            val charAsNum = char.digitToIntOrNull()
            val lastNumber = list.lastOrNull()

            fun completePartial(lastNumber: GridNumber) = list.replaceLast(lastNumber.copy(partial = false))
            fun startNewPartial(num: Int) = list + GridNumber(point, point, num, partial = true)
            fun appendToPartial(lastNumber: GridNumber) =
                list.replaceLast(lastNumber.copy(end = point, value = lastNumber.value.append(char)))

            when {
                lastNumber != null && lastNumber.partial ->
                    if (charAsNum != null) {
                        if (point.y == lastNumber.start.y) {
                            appendToPartial(lastNumber)
                        } else {
                            completePartial(lastNumber)
                            startNewPartial(charAsNum)
                        }
                    } else {
                        completePartial(lastNumber)
                    }

                charAsNum != null -> startNewPartial(charAsNum)
                else -> list

            }

        }

    val gears: List<Point> = map.toList().mapNotNull { (point, char) -> if (char == '*') point else null }

    private fun Int.append(char: Char) = (toString() + char.toString()).toInt()
    private fun <T> List<T>.replaceLast(value: T) = dropLast(1) + value
}

In [291]:
val map = input
    .lines()
    .withIndex()
    .flatMap { (y, line) -> line.withIndex().map { (x, char) -> Point(x, y) to char } }
    .toMap()

val Char.isValidSymbol get() = this != '.' && this !in '0'..'9'

val grid = Grid(map)
grid
    .numbers
    .filter { gridNumber ->
        gridNumber
            .adjacentPoints
            .mapNotNull { point -> grid[point].takeIf { it.isValidSymbol } }
            .isNotEmpty()
    }
    .map(GridNumber::value)
    .sum()
    


// 550064 is correct!

550064

# Part Two

In [292]:
grid
    .gears
    .map { gear -> 
        val numbers = grid.numbers.filter { number -> 
            gear.adjacentPoints.any { point -> number.overlaps(point) } 
        }
        if (numbers.size == 2) {
            val (first, second) = numbers
            first.value * second.value
        } else {
            0
        }
     }
     .sum()

85010461

# Puzzle Input

## Sample Input

In [272]:
val input = """
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
""".trimIndent()

## New line starting edge case input

In [216]:
val input = """
.*34
34..
""".trimIndent()